# 2.2 Pandas, Basic Mapping

This section aims to provide new skills in python to handle structured "table" data. 

Learning outcome:
-   Manipulation of data frames (describing, filtering, ...) 
-   Learn about Lambda functions
-   Intro to datetime objects
-   Plotting data from data frames (histograms and maps)
-   Introduction to Plotly
-   Introduction to CSV & Parquet


We will work on several structured data sets: sensor metadata, seismic data product (earthquake catalog).

First, we import all the modules we need:

In [ ]:
import numpy as np
import pandas as pd
import io
import requests
import time
from datetime import datetime, timedelta

import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'vscode' # writes as standalone html, 
# try notebook, jupyterlab, png, vscode, iframe


## Creating a data frame from standard text files

The python package pandas is very useful to read csv files, but also many text files that are more or less formated as one observation per row and one column for each feature.

As an example, we are going to look at the list of seismic stations from the Northern California seismic network, available [here](http://ncedc.org/ftp/pub/doc/NC.info/NC.channel.summary.day):



In [ ]:
url = 'http://ncedc.org/ftp/pub/doc/NC.info/NC.channel.summary.day'

The function read_csv is used to open and read your text file. In the case of a well formatted csv file, only the name of the file needs to be entered:

     data = pd.read_csv('my_file.csv')

However, many options are available if the file is not well formatted. See more in this [tutorial](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html):



In [ ]:
# this gets the file linked in the URL page and convert it to a string
s = requests.get(url).content 
# this will convert the string, decode it , and make it a table
data = pd.read_csv(io.StringIO(s.decode('utf-8')), header=None, skiprows=2, sep='\s+', usecols=list(range(0, 13)))
# because columns/keys were not assigned, assign them now
data.columns = ['station', 'network', 'channel', 'location', 'rate', 'start_time', 'end_time', 'latitude', 'longitude', 'elevation', 'depth', 'dip', 'azimuth']

Let us look at the data. They are now stored into a pandas dataframe. Read the top of the table:

In [ ]:
data.head()

There are two aways of looking at a particular column:

In [ ]:
data.station

In [ ]:
# or like this:
data['station']

If we want to look at a given row or column, and we know its index, we can do:

In [ ]:
data.iloc[0]

In [ ]:
data.iloc[:, 0]

If we know the name of the column, we can do:

In [ ]:
data.loc[:, 'station']

We can also access a single value within a column:

In [ ]:
data.loc[0, 'station']

## Mapping usiny Plotly

Now we will plot the station locations on a map using the Plotly package. More tutorials on [Plotly](https://plotly.com/). Input of the function in the function is self-explanatory and typical of Python's function. The code [documentation](https://plotly.com/python/scatter-plots-on-maps/) of Plotly scatter_geo lists the variables.

In [ ]:
fig = px.scatter_geo(data,
                     lat='latitude',lon='longitude', 
                     range_color=(6,9),
                     height=800, width=800,
                     hover_name="station",
                     hover_data=['network','station','channel','rate']);
fig.update_geos(resolution=110, showcountries=True,projection_type="orthographic")

In [ ]:
fig = px.scatter_mapbox(data,
                     lat='latitude',lon='longitude', 
                     range_color=(6,9),mapbox_style="carto-positron",
                     height=800, width=800,
                     hover_name="station",
                     hover_data=['network','station','channel','rate']);
fig.update_layout(title="Northern California Seismic Network")
fig.show()

## Pandas: data selection
We can filter the data with the value taken by a given column:

In [ ]:
data.loc[data.station=='KCPB']

In [ ]:
# Select two stations
data.loc[(data.station=='KCPB') | (data.station=='KHBB')]

In [ ]:
# or like this
data.loc[data.station.isin(['KCPB', 'KHBB'])]

We can access to a brief summary of the data:

In [ ]:
data.station.describe()

In [ ]:
data.elevation.describe()

We can perform standard operations on the whole data set:

In [ ]:
data.mean()

In the case of a categorical variable, we can get the list of possile values that this variable can take:

In [ ]:
data.channel.unique()

and get the number of times that each value is taken:

In [ ]:
data.station.value_counts()

There are several ways of doing an operation on all rows of a column. The first option is to use the map function.

If you are not familiar with lambda function in Python, look at:

https://realpython.com/python-lambda/

We will practice a bit lambda functions

In [ ]:
# Let's create a function to return the difference between the input and a value

b = 1 # b is declared outside of a function and is a GLOBAL variable

def remove_elevation(x):
    return x-b
print(remove_elevation(3))

In [ ]:
# now we are creating a function with a local variable
def remove_elevation2(x):
    b2=2 # b is now declared inside the function, is LOCAL, and cannot be called outside.
    return x-b2
print(remove_elevation2(3))
# print(b2)

In [ ]:
# Now the equivalent in lambda is:
remove_b = lambda x:x-b
 # b was defined above so it is a parameter. x is the variable

In [ ]:
remove_b(3)

In [ ]:
# you can add several variables into lambda functions
remove_anything = lambda x,y:x-y
remove_anything(3,b)

In [ ]:
data_elevation_mean = data.elevation.mean()
data.elevation.map(lambda p: p - data_elevation_mean)

The second option is to use the apply function:

In [ ]:
def remean_elevation(row):
    row.elevation = row.elevation - data_elevation_mean
    return row
data.apply(remean_elevation, axis='columns')

We can also carry out simple operations on columns, provided they make sense.

In [ ]:
netsta=(data.network + '.' + data.station)
print(netsta)
print(type(netsta))

In [ ]:
netsta=(data.network + '.' + data.station).values
print(netsta)
print(type(netsta))

A useful feature is to group the rows depending on the value of a categorical variable, and then apply the same operation to all the groups. For instance, we want to know how many times each station appears in the file:

In [ ]:
data.groupby('station').station.count()

Or we want to know what is the lowest and the highest elevation for each station:

In [ ]:
data.groupby('station').elevation.min()

We can have access to the data type of each column:

In [ ]:
data.dtypes

Here, pandas does not recognize the start_time and end_time columns as a datetime format, so we cannot use datetime operations on them. We first need to convert these columns into a datetime format:

In [ ]:
data.start_time.values()

In [ ]:
type(data['start_time'][0])

In [ ]:
# Transform column from string into datetime format
startdate = pd.to_datetime(data['start_time'], format='%Y/%m/%d,%H:%M:%S')
data['start_time'] = startdate
print(data['start_time'] )
type(data['start_time'][0])

In [ ]:
print(data['end_time'])

In [ ]:
# do the same for end times
# Avoid 'OutOfBoundsDatetime' error with year 3000
enddate = data['end_time'].str.replace('3000', '2025')
enddate = pd.to_datetime(enddate, format='%Y/%m/%d,%H:%M:%S')
data['end_time'] = enddate

⚠️ Advanced exercise: You may notice that some stations have a latitude and longitude of zero. Clean up the data frame and report it to the class. 

In [ ]:
# enter your answers here.

We can now look when each seismic station was installed:

In [ ]:
data.groupby('station').apply(lambda df: df.start_time.min())

The ``agg`` function allows to carry out several operations to each group of rows:

Select the stations that were deployed first and recovered last

In [ ]:
data.groupby(['station']).agg({'start_time':lambda x: min(x), 'end_time':lambda x: max(x)})

We can also make groups by selecting the values of two categorical variables:

In [ ]:
data.groupby(['station', 'channel']).agg({'start_time':lambda x: min(x), 'end_time':lambda x: max(x)})

Previously, we just printed the output, but we can also store it in a new variable:

In [ ]:
data_grouped = data.groupby(['station', 'channel']).agg({'start_time':lambda x: min(x), 'end_time':lambda x: max(x)})

Print the new dataframe and look at the rows indexes. Anything wrong?

In [ ]:
# enter code here

When we select only some rows, the index is not automatically reset to start at 0. We can do it manually. Many functions in pandas have also an option to reset the index, and option to transform the dataframe in place, instead of saving the results in another variable.

In [ ]:
data_grouped.reset_index()

It is also possible to sort the dataset by value.

In [ ]:
data_grouped.sort_values(by='start_time')

We can apply the sorting to several columns:

In [ ]:
data_grouped.sort_values(by=['start_time', 'end_time'])

The ``merge`` function allows to merge two dataframes that have some but not all columns in common.


Now, we are going to merge this seismic metadata frame to the one from the Pacific Northwest Seismic Network and other stations that were deployed in the area. The file is a CSV file. Read the file and describe the data



In [ ]:
# enter response here
data_pnw=pd.read_csv('PNSN_meta.csv')

In [ ]:
data_pnw.head()

You will notice that this file contains odd times for starttime and end times. The keys are also different from the previous dataframe. In order to compare them and concatenate both metadata, we want to at least have common keys.

First, make sure that both dataframes have common keys. Use the function [``rename``](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html) from ``pandas`` to rename the keys in data_pnw.

In [ ]:
# enter answer below

Then convert the time stamps of start_time and end_time into date time objects. 

In [ ]:
# Let's work together (i could not fix it!)

# L=len(data_pnw['starttime'])
# newdt = np.ndarray(L,dtype=np.datetime64)
# for icrap in range(len(data_pnw['starttime'])):
#     print(icrap)
#     print(type(data_pnw['starttime'][icrap]))
#     print(data_pnw['starttime'][icrap])
#     newdt[icrap]= datetime.fromtimestamp(data_pnw['starttime'][icrap])
# print(newdt)
# print(data_pnw.starttime)

# startdate = pd.to_timedelta(data_pnw['starttime'])
# startdate = pd.to_timedelta(datetime.fromtimestamp(data_pnw.starttime), format='%Y/%m/%d,%H:%M:%S')
# print(startdate)

In [ ]:
df = pd.concat([data,data_pnw],ignore_index=True)

In [ ]:
# Earthquakes in filtered 2007-2009 catalog but not in (unfiltered) 2004-2011 catalog
df_all = pd.concat([df2, df1_filter], ignore_index=True)
df_merge = df_all.merge(df2.drop_duplicates(), on=['date'], how='left', indicator=True)
df_added_1 = df_merge[df_merge['_merge'] == 'left_only']

# Earthquakes in filtered 2004-2011 catalog but not in (unfiltered) 2007-2009 catalog
df_all = pd.concat([df1, df2_filter], ignore_index=True)
df_merge = df_all.merge(df1.drop_duplicates(), on=['date'], how='left', indicator=True)
df_added_2 = df_merge[df_merge['_merge'] == 'left_only']

### Overview of Earthquake catalog

In [ ]:
quake=pd.read_csv('Global_Quakes_IRIS.csv')
quake.head()

* Plot the earthquakes using Plotly functionality.

Use "description" as a hover name. 

Use "marker_size" for size and color the markers using the key 'magnitude'.

Use "magnitude" and "depth" as hover data.

In [ ]:
# enter answer here

Using ``matplotlib``, make a histogram of the magnitude distribution of these earthquakes. Find a way to plot the y-axis in log scale.

In [ ]:
# enter answer here